In [ ]:
#pip install openai
#pip install langchain
#pip install wikipedia

In [ ]:
import openai
import os
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.agents.react.base import DocstoreExplorer
from langchain.docstore import Wikipedia
docstore = DocstoreExplorer(Wikipedia())

In [ ]:
from langchain.agents import Tool 
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4")

In [ ]:
from langchain.agents import AgentType, initialize_agent
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [ ]:
question = "What profession does Amitabh Bachchan and Shahrukh Khan have in common?"
result = react.run(question)

In [ ]:
result